In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle

In [2]:
%%time
train_df = pd.read_csv("./data/training.csv")

CPU times: user 2.46 s, sys: 409 ms, total: 2.87 s
Wall time: 3.08 s


In [3]:
routes_df = pd.read_csv("./data/routes.csv")
route_list = routes_df['BUSROUTE_ID'].unique()

stop_df = pd.read_csv("./data/stops.csv")
stop_list = stop_df['BUSSTOP_ID'].unique()

bus_list = train_df['BUS_ID'].unique()

In [4]:

# routes_df['seq_max2'] = routes_df.groupby(['BUSROUTE_ID', 'BUSSTOP_ID'])['BUSSTOP_SEQ'].transform(max)
route_max = routes_df.groupby(['BUSROUTE_ID'])['BUSSTOP_SEQ'].max()
pickle.dump(route_max.to_dict()  , open("./out/route_max.pickle"    , "wb"))

In [4]:
train_df = train_df.sort_values(by=['BUSROUTE_ID', 'BUS_ID', 'TIMESTAMP'])

In [ ]:
%%time
route_index = 0
prev_seq = 0
seq_data = []
for idx, row in train_df.iterrows():
    if prev_seq == 0:
        prev_bus = row['BUS_ID']
        prev_seq = row['BUSSTOP_SEQ']
        prev_route = row['BUSROUTE_ID']
        prev_timestamp = row['TIMESTAMP']
        min_timestamp = row['TIMESTAMP']
        cnt = 1
        continue
        
    if (row['BUSSTOP_SEQ']>prev_seq and row['BUS_ID'] == prev_bus and row['BUSROUTE_ID'] == prev_route):
        pass
    else:
        seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])
        min_timestamp = row['TIMESTAMP']
        route_index+=1
        cnt = 0

        
        
    prev_bus = row['BUS_ID']
    prev_seq = row['BUSSTOP_SEQ']
    prev_route = row['BUSROUTE_ID']
    prev_timestamp = row['TIMESTAMP']
    train_df.at[idx, 'ROUTE_IDX'] = route_index
    cnt += 1
    
seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])

In [20]:
%%time
train_df_bk = train_df
train_df=train_df.fillna(0)
for idx, row in train_df.iterrows():
    train_df.at[idx, 'ROUTE_TIME'] = seq_data[int(row['ROUTE_IDX'])][3]
    train_df.at[idx, 'SEQ_NUM'] = seq_data[int(row['ROUTE_IDX'])][2]

CPU times: user 9min 37s, sys: 6.86 s, total: 9min 43s
Wall time: 10min 29s


In [8]:
train_df['TIMESTAMP_DIFF'] = train_df['TIMESTAMP'].shift(-1) - train_df['TIMESTAMP']
train_df['SEQ_DIFF'] = train_df['BUSSTOP_SEQ'].shift(-1) - train_df['BUSSTOP_SEQ']
train_df['ROUTE_DIFF'] = train_df['BUSROUTE_ID'].shift(-1) - train_df['BUSROUTE_ID']
train_df['BUS_ID_DIFF'] = train_df['BUS_ID'].shift(-1) - train_df['BUS_ID']

In [11]:
train_df['TIMESTAMP_DIFF_REV'] = train_df['TIMESTAMP'] - train_df['TIMESTAMP'].shift(1)
train_df['SEQ_DIFF_REV'] = train_df['BUSSTOP_SEQ'] - train_df['BUSSTOP_SEQ'].shift(1)

train_df['ROUTE_DIFF_REV'] = train_df['BUSROUTE_ID'].shift(-1) - train_df['BUSROUTE_ID']
train_df['BUS_ID_DIFF_REV'] = train_df['BUS_ID'].shift(-1) - train_df['BUS_ID']

In [12]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,TIMESTAMP_DIFF_2,TIMESTAMP_DIFF_REV,SEQ_DIFF_REV
173467,173467,2020-01-06 17:03:34,2053,11100010,380,21,1578330214,570291.0,-19.0,0.0,0.0,NaN,NaN,NaN
1624542,1624542,2020-01-13 07:28:25,2053,11100010,18,2,1578900505,117.0,1.0,0.0,0.0,570291.0,570291.0,-19.0
1625151,1625151,2020-01-13 07:30:22,2053,11100010,11,3,1578900622,123.0,1.0,0.0,0.0,117.0,117.0,1.0
1625791,1625791,2020-01-13 07:32:25,2053,11100010,9,4,1578900745,121.0,1.0,0.0,0.0,123.0,123.0,1.0
1626425,1626425,2020-01-13 07:34:26,2053,11100010,6,5,1578900866,136.0,1.0,0.0,0.0,121.0,121.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614825,1614825,2020-01-12 22:29:06,889203038,11400130,9,10,1578868146,185.0,1.0,0.0,0.0,123.0,123.0,1.0
1614944,1614944,2020-01-12 22:32:11,889203038,11400130,6,11,1578868331,99.0,1.0,0.0,0.0,185.0,185.0,1.0
1614996,1614996,2020-01-12 22:33:50,889203038,11400130,4,12,1578868430,225.0,1.0,0.0,0.0,99.0,99.0,1.0
1615093,1615093,2020-01-12 22:37:35,889203038,11400130,147,13,1578868655,150.0,1.0,0.0,0.0,225.0,225.0,1.0


In [28]:
# 1 route-iig 3 tsagaas ih hugatsaand tuulsan bol zogsolt hiisen gej uzeed hasalt hiisen
# train_df[(train_df['ROUTE_TIME']>2) & (train_df['ROUTE_TIME']<3)].head(60)
train_df = train_df[train_df['ROUTE_TIME']<3]

In [ ]:
train_df = train_df[train_df['SEQ_DIFF']==1]
# train_df = train_df[train_df['SEQ_DIFF']>0]
train_df = train_df[train_df['ROUTE_DIFF']==0]
train_df = train_df[train_df['BUS_ID_DIFF']==0]

In [29]:
# 3220254 -> 2969021 -> 2871406
train_df.head()

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF
173467,173467,2020-01-06 17:03:34,2053,11100010,380,21,1578330214,0.0,0.000000,1.0,570291.0,-19.0,0.0,0.0
1624542,1624542,2020-01-13 07:28:25,2053,11100010,18,2,1578900505,1.0,0.817500,21.0,117.0,1.0,0.0,0.0
1625151,1625151,2020-01-13 07:30:22,2053,11100010,11,3,1578900622,1.0,0.817500,21.0,123.0,1.0,0.0,0.0
1625791,1625791,2020-01-13 07:32:25,2053,11100010,9,4,1578900745,1.0,0.817500,21.0,121.0,1.0,0.0,0.0
1626425,1626425,2020-01-13 07:34:26,2053,11100010,6,5,1578900866,1.0,0.817500,21.0,136.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614825,1614825,2020-01-12 22:29:06,889203038,11400130,9,10,1578868146,89583.0,0.903056,13.0,185.0,1.0,0.0,0.0
1614944,1614944,2020-01-12 22:32:11,889203038,11400130,6,11,1578868331,89583.0,0.903056,13.0,99.0,1.0,0.0,0.0
1614996,1614996,2020-01-12 22:33:50,889203038,11400130,4,12,1578868430,89583.0,0.903056,13.0,225.0,1.0,0.0,0.0
1615093,1615093,2020-01-12 22:37:35,889203038,11400130,147,13,1578868655,89583.0,0.903056,13.0,150.0,1.0,0.0,0.0


In [32]:
# Dutuu route yavsan bol hasna.
train_df = pd.merge(train_df, route_max, on='BUSROUTE_ID', how='inner')
train_df = train_df[1.1*train_df['SEQ_NUM']>=train_df['BUSSTOP_SEQ_y']-1]

In [34]:
# Ali neg buudal deer 2000second-ees udaan zogsolt hiisen bol ter route-iig hassan
route_idx_max = train_df.groupby(['ROUTE_IDX'])['TIMESTAMP_DIFF'].max()
train_df = pd.merge(train_df, route_idx_max, on='ROUTE_IDX', how='inner')
train_df = train_df[train_df['TIMESTAMP_DIFF_y']<=2000]
train_df = train_df.rename(columns={"TIMESTAMP_DIFF_x": "TIMESTAMP_DIFF"})

In [39]:
train_df.head()

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ_x,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,BUSSTOP_SEQ_y,TIMESTAMP_DIFF_y
0,159011,2020-01-06 16:06:45,2061,11100010,6,5,1578326805,6.0,2.175556,36.0,120.0,1.0,0.0,0.0,40,815.0
1,159502,2020-01-06 16:08:45,2061,11100010,4,6,1578326925,6.0,2.175556,36.0,146.0,1.0,0.0,0.0,40,815.0
2,160153,2020-01-06 16:11:11,2061,11100010,147,7,1578327071,6.0,2.175556,36.0,124.0,1.0,0.0,0.0,40,815.0
3,160665,2020-01-06 16:13:15,2061,11100010,149,8,1578327195,6.0,2.175556,36.0,179.0,1.0,0.0,0.0,40,815.0
4,161461,2020-01-06 16:16:14,2061,11100010,151,9,1578327374,6.0,2.175556,36.0,217.0,1.0,0.0,0.0,40,815.0


In [42]:
mean_route_time = train_df.groupby(['BUSROUTE_ID'])['ROUTE_TIME'].mean().to_dict()
pickle.dump(mean_route_time  , open("./out/mean_route_time.pickle"    , "wb"))

# Tseverlegeegeer 10 shirheg route hasagdsan. Ene tohioldold niit dundajaar route_time-iig avna
default_route_time = train_df['ROUTE_TIME'].mean()
default_route_time

In [161]:
# bus id encoding
bus_id_encoder = preprocessing.LabelEncoder()
bus_id_encoder.fit(bus_list)
train_df.BUS_ID = bus_id_encoder.transform(train_df.BUS_ID)

# bus_id_encoder.transform([145010001, 235501055])
# bus_id_encoder.inverse_transform([280, 466])

In [162]:
# route id encoding
route_id_encoder = preprocessing.LabelEncoder()
route_id_encoder.fit(route_list)
train_df.BUSROUTE_ID = route_id_encoder.transform(train_df.BUSROUTE_ID)

In [163]:
# stop id encoding
stop_id_encoder = preprocessing.LabelEncoder()
stop_id_encoder.fit(stop_list)
train_df.BUSSTOP_ID = stop_id_encoder.transform(train_df.BUSSTOP_ID)

In [164]:
pickle.dump(bus_id_encoder  , open("./out/bus_id_encoder.pickle"    , "wb"))
pickle.dump(route_id_encoder, open("./out/route_id_encoder.pickle"  , "wb"))
pickle.dump(stop_id_encoder , open("./out/busstop_id_encoder.pickle", "wb"))

In [166]:
train_df['DAY_OF_WEEK'     ] = pd.to_datetime(train_df['RECORD_DATE']).dt.dayofweek
train_df['HOUR_OF_DAY'     ] = pd.to_datetime(train_df['RECORD_DATE']).dt.hour
train_df['MINUTE_OF_HOUR'  ] = pd.to_datetime(train_df['RECORD_DATE']).dt.minute
train_df['SECOND_OF_MINUTE'] = pd.to_datetime(train_df['RECORD_DATE']).dt.second

In [167]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ_x,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,TIMESTAMP_DIFF_x,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,BUSSTOP_SEQ_y,TIMESTAMP_DIFF_y,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE
0,159011,2020-01-06 16:06:45,2,3,5,5,1578326805,6.0,2.175556,36.0,120.0,1.0,0.0,0.0,40,815.0,0,16,6,45
1,159502,2020-01-06 16:08:45,2,3,3,6,1578326925,6.0,2.175556,36.0,146.0,1.0,0.0,0.0,40,815.0,0,16,8,45
2,160153,2020-01-06 16:11:11,2,3,146,7,1578327071,6.0,2.175556,36.0,124.0,1.0,0.0,0.0,40,815.0,0,16,11,11
3,160665,2020-01-06 16:13:15,2,3,148,8,1578327195,6.0,2.175556,36.0,179.0,1.0,0.0,0.0,40,815.0,0,16,13,15
4,161461,2020-01-06 16:16:14,2,3,150,9,1578327374,6.0,2.175556,36.0,217.0,1.0,0.0,0.0,40,815.0,0,16,16,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468868,1606842,2020-01-12 21:14:57,995,271,20,22,1578863697,89582.0,1.656944,26.0,253.0,1.0,0.0,0.0,27,1089.0,6,21,14,57
2468869,1607523,2020-01-12 21:19:10,995,271,36,23,1578863950,89582.0,1.656944,26.0,196.0,1.0,0.0,0.0,27,1089.0,6,21,19,10
2468870,1608043,2020-01-12 21:22:26,995,271,1224,24,1578864146,89582.0,1.656944,26.0,1089.0,1.0,0.0,0.0,27,1089.0,6,21,22,26
2468871,1610752,2020-01-12 21:40:35,995,271,841,25,1578865235,89582.0,1.656944,26.0,197.0,1.0,0.0,0.0,27,1089.0,6,21,40,35


In [171]:
# time validation (valid time is 06:00:00 ~ 23:59:59)
starting_time = 21600 # 06:00:00
train_df['TIME_validation'] = train_df['TIMESTAMP'] % 86400
train_df = train_df[train_df['TIME_validation']>=starting_time]

In [172]:
# encodeing time

# (pd.to_datetime(train_df['RECORD_DATE']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# (pd.Timestamp('2020-01-14 07:00:00')  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train_df['TIME'] = (train_df['TIMESTAMP'] - starting_time) % 86400


In [180]:
train_df.sort_values(by=['TIMESTAMP_DIFF'])
# train_df[train_df['ROUTE_IDX']==53245.0]

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ_x,TIMESTAMP,ROUTE_IDX,ROUTE_TIME,SEQ_NUM,...,ROUTE_DIFF,BUS_ID_DIFF,BUSSTOP_SEQ_y,TIMESTAMP_DIFF_y,DAY_OF_WEEK,HOUR_OF_DAY,MINUTE_OF_HOUR,SECOND_OF_MINUTE,TIME_validation,TIME
2160846,2495182,2020-01-16 13:20:24,610,206,378,16,1579180824,74618.0,1.865833,29.0,...,0.0,0.0,31,1474.0,3,13,20,24,48024,26424
418012,1170937,2020-01-10 14:41:21,427,41,452,40,1578667281,14090.0,0.921389,40.0,...,0.0,0.0,43,341.0,4,14,41,21,52881,31281
440639,760531,2020-01-08 21:14:36,522,41,1166,4,1578518076,14717.0,1.136944,42.0,...,0.0,0.0,43,393.0,2,21,14,36,76476,54876
419030,2027572,2020-01-14 16:26:16,427,41,453,42,1579019176,14118.0,1.016944,42.0,...,0.0,0.0,43,381.0,1,16,26,16,59176,37576
418909,1974295,2020-01-14 13:25:18,427,41,451,3,1579008318,14116.0,0.886667,42.0,...,0.0,0.0,43,355.0,1,13,25,18,48318,26718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210646,1500418,2020-01-12 12:23:35,798,119,434,56,1578831815,41905.0,2.263889,56.0,...,0.0,0.0,57,1998.0,6,12,23,35,44615,23015
1046862,3023853,2020-01-18 18:44:13,339,104,153,41,1579373053,37196.0,2.874722,79.0,...,0.0,0.0,80,1998.0,5,18,44,13,67453,45853
2175583,1824444,2020-01-13 19:10:29,499,207,286,13,1578942629,75372.0,2.123611,19.0,...,0.0,0.0,20,2000.0,0,19,10,29,69029,47429
1080849,3185030,2020-01-19 18:55:19,566,108,408,10,1579460119,38117.0,2.888611,33.0,...,0.0,0.0,34,2000.0,6,18,55,19,68119,46519


In [ ]:
# 2000 sec-ees ihiig evdersen esvel tur zogsolt hiisen gej uzeed haslaa

# train_df[(train_df['TIMESTAMP_DIFF']>2000)].sort_values(by=['TIMESTAMP_DIFF'])
# train_df = train_df[(train_df['TIMESTAMP_DIFF']<2000)]

In [181]:
output_df = train_df[[
    "DAY_OF_WEEK", "HOUR_OF_DAY", "MINUTE_OF_HOUR", "SECOND_OF_MINUTE", "TIME", 
    "BUS_ID", "BUSROUTE_ID", "BUSSTOP_ID", "TIMESTAMP_DIFF", "ROUTE_TIME", "SEQ_NUM"
]]

In [182]:
output_df.to_csv('./out/cleaned_train2.csv', index = False)